

#**Case Study 3.1**
In this case study, you will perform essential data preprocessing steps on the Penguins dataset. The dataset contains information about different species of penguins, including their physical characteristics and the region where they were observed. Your goal is to prepare the dataset for machine learning analysis. Follow these steps:
1. Load the penguins dataset using the code snippet provided below.
2. Perform initial data exploration to understand the dataset's structure, features, and any missing values. Summarize the dataset's statistics and gain insights into the data.
3. Address any data quality issues, such as missing values and outliers. Decide on an appropriate strategy for handling missing data, such as imputation or removal of rows/columns.
4. Analyze the relevance of each feature for your machine learning task by using the learned use feature selection technques.
5. If the dataset contains categorical variables, encode them into a numerical format suitable for machine learning models.
6. Split the dataset into training and testing subsets to evaluate the performance of your machine learning models.
7. Scale or normalize the numerical features to ensure consistent scaling across variables.
8. Apply suitable dimensionality reduction techniques to reduce the size of the data while preserving important information.
9. Validate your preprocessing pipeline by training and evaluating a machine learning model, such as the Random Forest model, on the preprocessed data. Compare the results to the model trained on the raw data (before feature filtering, transformation, and reduction) to ensure that preprocessing has improved model performance.








In [ ]:
import pandas as pd
from seaborn import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score

# Step 1: Load the penguins dataset
print("===================================================================================")
print("Step 1: Load the penguins dataset")
print("===================================================================================")
df = load_dataset('penguins')
print(df.head())

# Step 2: Initial data exploration
print("===================================================================================")
print("Step 2: Initial data exploration")
print("===================================================================================")

print("Dataset Info:")
print(df.info())

# Check for missing values
print("\nMissing Values:")
missing_values = df.isnull().sum()
print(missing_values)

# Summary statistics
print("\nSummary Statistics:")
summary_stats = df.describe()
print(summary_stats)

# Step 3: Handle missing values
print("===================================================================================")
print("Step 3: Handle missing values")
print("===================================================================================")

# Impute missing values
df['species'].fillna(df['species'].mode()[0], inplace=True)
df['island'].fillna(df['island'].mode()[0], inplace=True)
numerical_columns = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
for column in numerical_columns:
    df[column].fillna(df[column].mean(), inplace=True)
df['sex'].fillna(df['sex'].mode()[0], inplace=True)

# Check if missing values handled
missing_values = df.isnull().sum()
print("\nMissing Values after Imputation:")
print(missing_values)

# Step 4: Feature selection using VarianceThreshold
print("===================================================================================")
print("Step 4: Feature selection using VarianceThreshold")
print("===================================================================================")

# Select numerical values
numerical_columns = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

df_numerical = df[numerical_columns]

# Initialize the threshold
selector = VarianceThreshold(threshold=0.001)

# Fit the selector to numerical data
df_high_variance = selector.fit_transform(df_numerical)

print("Original Numerical Data:")
print(df_numerical)

print("Numerical Data after VarianceThreshold:")
print(df_high_variance)

# Step 5: Encode categorical variables using OneHotEncoder
print("===================================================================================")
print("Step 5: Encode categorical variables using OneHotEncoder")
print("===================================================================================")

# Initialize the encoder
enc = OneHotEncoder(handle_unknown='ignore')

# Columns that want to make one-hot encode of them
columns_to_encode = ['species', 'island', 'sex']

enc.fit(df[columns_to_encode])

# First, apply one-hot encoding to the arrays
# Then, converting it into dense arrays
df_encoded = enc.transform(df[columns_to_encode]).toarray()

# Copy the original DataFrame
# The copy will be used for storing encoded data
df_ohenc = df.copy()

# Adding them to the new DataFrame
encoded_columns = [f'{col}_{value}' for col, values in zip(columns_to_encode, enc.categories_) for value in values]
df_ohenc[encoded_columns] = df_encoded

print(df_ohenc.head())

# Step 6: Split the dataset into training and testing subsets
print("\nStep 6: Split the dataset into training and testing subsets")
print("===================================================================================")

# Without the target variable
X = df_ohenc.drop('species_Gentoo', axis=1)

# Target
y = df_ohenc['species_Gentoo']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Remove categorical features from X_train and X_test
X_train = X_train.drop(columns=['species', 'island', 'sex'])
X_test = X_test.drop(columns=['species', 'island', 'sex'])


# Step 7: Scale or normalize numerical features using MinMaxScaler
print("===================================================================================")
print("Step 7: Scale or normalize numerical features using MinMaxScaler")
print("===================================================================================")

# Select just one numerical columns to scale it
numerical_columns = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

df_numerical = df_ohenc[numerical_columns]

# Initialize the scaler
scaler = MinMaxScaler()

# Fitting it to the training data
scaler.fit(X_train[numerical_columns])

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_columns] = scaler.transform(X_train[numerical_columns])
X_test_scaled[numerical_columns] = scaler.transform(X_test[numerical_columns])

print("Summary Statistics of Scaled Training Data:")
print(X_train_scaled.describe())

print("\nSummary Statistics of Scaled Testing Data:")
print(X_test_scaled.describe())

# Step 8: Apply dimensionality reduction (PCA)
print("===================================================================================")
print("Step 8: Apply dimensionality reduction (PCA)")
print("===================================================================================")

# Select the numerical feautures
numerical_columns = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

# Make an instance of PCA
pca = PCA(n_components=2)

# Scale the training data (without one-hot encoded columns)
# Fitting PCA model on this scaled dataset
pca.fit(X_train_scaled[numerical_columns])
X_train_pca = pca.transform(X_train_scaled[numerical_columns])
X_test_pca = pca.transform(X_test_scaled[numerical_columns])

# Create new DataFrame
df_PCA = pd.DataFrame(X_train_pca, columns=['PCA_1', 'PCA_2'])

print("Data after PCA:")
print(df_PCA.head())

# Step 9: Train a machine learning model (Random Forest) on the preprocessed data
print("===================================================================================")
print("Step 9: Train a machine learning model (Random Forest) on the preprocessed data")
print("===================================================================================")
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_pca, y_train)
# Evaluate the testing data
y_pred = rf_model.predict(X_test_pca)
# Accuracy for evaluation the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy with Preprocessed Data:", accuracy)
print("Training and evaluating the model on raw data (before preprocessing)")
rf_model_raw = RandomForestClassifier(random_state=42)
# Train the RF model on raw data
rf_model_raw.fit(X_train, y_train)
# Evaluate the testing data
y_pred_raw = rf_model_raw.predict(X_test)
# Accuracy of model evaluation
accuracy_raw = accuracy_score(y_test, y_pred_raw)
print("\nAccuracy with Raw Data (before preprocessing):", accuracy_raw)
# Compare the performance
print("\nAccuracy Improvement:", accuracy - accuracy_raw)


Step 1: Load the penguins dataset
  species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
3  Adelie  Torgersen             NaN            NaN                NaN   
4  Adelie  Torgersen            36.7           19.3              193.0   

   body_mass_g     sex  
0       3750.0    Male  
1       3800.0  Female  
2       3250.0  Female  
3          NaN     NaN  
4       3450.0  Female  
Step 2: Initial data exploration
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 